In [24]:
import pandas as pd
import geopandas as gpd
import unidecode

import folium
from folium.features import GeoJsonTooltip

import json
import os

import sys
sys.path.append('../..')
from src.db.db_handler import DBHandler
from helpers.coropleth_map import unificar_geojson_provincias, merge_geojson_provincias, crear_mapa_choropleth

In [25]:
db = DBHandler()

In [26]:
fecha_temp = db.get_latest_historical_date("temperatura_historico")
temp_data = db.get_estaciones_historico_rango("temperatura", fecha_temp, fecha_temp)

In [27]:
provincias = db.get_table("provincias")
estaciones = db.get_table("estaciones")

df_provincias = pd.DataFrame(provincias)
df_estaciones = pd.DataFrame(estaciones)

df_temp = pd.DataFrame(temp_data)                   

In [28]:
df_temp_merged = df_temp.merge(df_estaciones, on="idema", how="left")
df_temp_merged = df_temp_merged.merge(df_provincias, left_on="provincia_id", right_on="id", how="left", suffixes=("_est", "_prov"))

In [29]:
df_temp_merged = df_temp_merged.sort_values("fecha")

In [30]:
df_temp_prov = df_temp_merged.groupby("provincia_id", as_index=False).agg({
    "nombre_prov": "last",
    "tmed": "mean"
})


In [31]:
df_temp_prov["nombre_prov"] = df_temp_prov["nombre_prov"].apply(lambda x: unidecode.unidecode(x.strip().lower()))

In [32]:
geojson_dir = "../../data/geojson/provincias"

In [33]:
geojson_unificado = unificar_geojson_provincias(geojson_dir)
geojson_merged = merge_geojson_provincias(geojson_unificado)

Número total de features en el GeoJSON unificado: 57
Número de features después de fusionar: 52


In [34]:
df_temp_prov["provincia_id"] = df_temp_prov["provincia_id"].astype(str)

In [35]:
mapa = crear_mapa_choropleth(
    geojson_data=geojson_merged,
    df=df_temp_prov,
    id_col="provincia_id",
    value_col="tmed",
    tooltip_field=["nombre_prov", "tmed"],
    tooltip_alias=["Provincia:", "Temperatura media (°C):"],
    legend_name="Temperatura Media (°C)"
)

In [36]:
mapa

In [37]:
#mapa_temperaturas.save("mapa.html")